In [1]:
from numpy import load, zeros, ones
from numpy.random import randn, randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout

from matplotlib import pyplot

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="0"

/home/nibraas/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nibraas/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nibraas/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nibraas/anaconda3/envs/tf_gpu/lib/pyt

In [2]:
def define_discriminator(in_shape=(80,80,3)):
    model = Sequential()

    model.add(Conv2D(128, (5,5), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    print(model.summary())
    return model

In [3]:
def define_generator(latent_dim):
    model = Sequential()

    n_nodes = 128 * 5 * 5
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((5, 5, 128)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (5,5), activation='tanh', padding='same'))
    return model

In [4]:
def define_gan(g_model, d_model):
    d_model.trainable = False

    model = Sequential()

    model.add(g_model)

    model.add(d_model)

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    print(model.summary())
    return model

In [5]:
def load_real_samples(num_of_samples):
    data = load('img_align_celeba.npz')
    X = data['arr_0'][num_of_samples:num_of_samples+100000]
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    return X

In [6]:
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y

In [7]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [8]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

In [9]:
def save_plot(examples, epoch, n=10):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(examples[i])
    filename = '/home/nibraas/Coding/GenerativeModels/img_align_celeba_generated/generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

In [10]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    save_plot(x_fake, epoch)
    filename = '/home/nibraas/Coding/GenerativeModels/img_align_celeba_models/generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))

        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [12]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples(50000)
train(g_model, d_model, gan_model, dataset, latent_dim)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 128)       9728      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 80, 80, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       409728    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 40, 40, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       409728    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 128)       409728    
__________

>1, 16/781, d1=0.000, d2=0.015 g=4.889
>1, 17/781, d1=0.000, d2=0.011 g=5.401
>1, 18/781, d1=0.000, d2=0.636 g=9.843
>1, 19/781, d1=0.000, d2=0.000 g=15.765
>1, 20/781, d1=0.000, d2=0.000 g=14.661
>1, 21/781, d1=0.000, d2=0.004 g=8.562
>1, 22/781, d1=0.000, d2=0.939 g=16.118
>1, 23/781, d1=2.730, d2=0.000 g=14.610
>1, 24/781, d1=0.159, d2=0.025 g=4.151
>1, 25/781, d1=0.068, d2=9.010 g=8.015
>1, 26/781, d1=0.046, d2=0.001 g=8.909
>1, 27/781, d1=0.056, d2=0.032 g=4.853
>1, 28/781, d1=0.030, d2=0.355 g=3.791
>1, 29/781, d1=0.065, d2=0.210 g=3.385
>1, 30/781, d1=0.114, d2=0.495 g=5.264
>1, 31/781, d1=0.921, d2=0.079 g=2.731
>1, 32/781, d1=0.023, d2=0.707 g=5.261
>1, 33/781, d1=0.033, d2=0.010 g=6.060
>1, 34/781, d1=0.056, d2=0.031 g=3.791
>1, 35/781, d1=0.013, d2=0.052 g=3.344
>1, 36/781, d1=0.001, d2=0.192 g=2.527
>1, 37/781, d1=0.005, d2=0.595 g=5.293
>1, 38/781, d1=8.895, d2=0.477 g=1.578


>1, 39/781, d1=0.000, d2=2.945 g=0.808
>1, 40/781, d1=0.000, d2=0.620 g=1.450
>1, 41/781, d1=0.000, d2=0.195 g=2.210
>1, 42/781, d1=0.387, d2=0.198 g=1.881
>1, 43/781, d1=0.063, d2=0.189 g=2.799
>1, 44/781, d1=0.000, d2=0.094 g=3.486
>1, 45/781, d1=0.000, d2=0.036 g=4.632
>1, 46/781, d1=0.000, d2=0.011 g=5.600
>1, 47/781, d1=0.009, d2=0.009 g=5.444
>1, 48/781, d1=0.000, d2=0.012 g=5.248
>1, 49/781, d1=0.001, d2=0.011 g=5.277
>1, 50/781, d1=0.012, d2=0.017 g=4.798
>1, 51/781, d1=0.001, d2=0.026 g=5.983
>1, 52/781, d1=0.093, d2=0.031 g=5.262
>1, 53/781, d1=0.009, d2=0.065 g=10.081
>1, 54/781, d1=0.067, d2=0.001 g=8.821
>1, 55/781, d1=0.017, d2=0.090 g=11.259
>1, 56/781, d1=0.049, d2=0.002 g=6.826
>1, 57/781, d1=0.060, d2=0.376 g=16.118
>1, 58/781, d1=1.566, d2=0.000 g=13.293
>1, 59/781, d1=0.054, d2=0.002 g=5.323
>1, 60/781, d1=0.000, d2=0.192 g=9.608
>1, 61/781, d1=0.087, d2=0.001 g=8.573


>1, 62/781, d1=0.000, d2=0.004 g=8.034
>1, 63/781, d1=0.000, d2=0.006 g=7.958
>1, 64/781, d1=0.000, d2=0.009 g=9.244
>1, 65/781, d1=0.148, d2=0.358 g=16.115
>1, 66/781, d1=1.387, d2=0.000 g=16.038
>1, 67/781, d1=0.362, d2=0.002 g=4.348
>1, 68/781, d1=0.067, d2=0.316 g=7.195
>1, 69/781, d1=0.000, d2=0.000 g=12.331
>1, 70/781, d1=0.000, d2=0.000 g=12.570
>1, 71/781, d1=0.253, d2=0.000 g=10.102
>1, 72/781, d1=0.208, d2=0.762 g=5.883
>1, 73/781, d1=0.469, d2=0.033 g=8.902
>1, 74/781, d1=0.410, d2=1.204 g=11.869
>1, 75/781, d1=1.408, d2=0.004 g=6.676
>1, 76/781, d1=1.061, d2=0.008 g=5.599
>1, 77/781, d1=1.064, d2=0.017 g=5.113


KeyboardInterrupt: 

In [ ]:
import numpy as np

In [ ]:
def plot_faces(faces, n):
    faces = (faces * 127.5 + 127.5).astype(np.uint8)
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(faces[i])
    pyplot.show()

In [ ]:
x_fake, y_fake = generate_fake_samples(g_model, latent_dim, 4)

In [ ]:
plot_faces(x_fake, 2)